In [ ]:
%cd /content/drive/MyDrive/Machine Learning

In [ ]:
%pip install git+https://github.com/Nixtla/neuralforecast.git

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import copy
from datetime import datetime
from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, PatchTST, iTransformer, TSMixer
from utilsforecast.losses import mae, mse
from utilsforecast.evaluation import evaluate


def load_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data = data[['Close', 'Volume']]  # Include trading volume as a feature
    data['Returns'] = data['Close'].pct_change()  # Calculate percentage returns
    data = data.dropna()  # Remove missing values
    data = data.reset_index()
    data.columns = ['ds', 'y', 'Volume', 'Returns']
    data['unique_id'] = ticker
    return data

def run_experiment(stock_data, val_size, test_size, freq, ticker, horizon, epochs):
    models = [
        iTransformer(h=horizon, input_size=5*horizon, n_series=1, max_steps=epochs, early_stop_patience_steps=5),
        TSMixer(h=horizon, input_size=5*horizon, n_series=1, max_steps=epochs, early_stop_patience_steps=5),
        NHITS(h=horizon, input_size=5*horizon, max_steps=epochs, early_stop_patience_steps=5),
        PatchTST(h=horizon, input_size=5*horizon, max_steps=epochs, early_stop_patience_steps=5)
    ]

    # Save the mean and std dev of the columns
    mean = stock_data[['y', 'Volume', 'Returns']].mean()
    std = stock_data[['y', 'Volume', 'Returns']].std()

    # Normalize the input data
    stock_data[['y', 'Volume', 'Returns']] = (stock_data[['y', 'Volume', 'Returns']] - mean) / std

    nf = NeuralForecast(models=models, freq=freq)
    nf_preds = nf.cross_validation(df=stock_data, val_size=val_size, test_size=test_size, n_windows=None)
    nf_preds = nf_preds.reset_index()
    evaluation = evaluate(df=nf_preds, metrics=[mae, mse], models=['iTransformer', 'TSMixer', 'NHITS', 'PatchTST'])
    evaluation.to_csv(f'{ticker}_results_horizon_{horizon}_epochs_{epochs}.csv', index=False, header=True)
    return evaluation, nf_preds

tickers = ['SPY']
start_date = '2010-01-01'
end_date = '2023-04-30'
horizon = 7
epochs = 2000

dataframes = []
predictions_dfs = []
for ticker in tickers:
    stock_data = load_stock_data(ticker, start_date, end_date)
    val_size = 365
    test_size = 7
    freq = 'D'

    evaluation, nf_preds = run_experiment(stock_data, val_size, test_size, freq, ticker, horizon, epochs)
    evaluation['ticker'] = ticker
    dataframes.append(evaluation)
    predictions_dfs.append(nf_preds.copy())


full_df = pd.concat(dataframes, ignore_index=True)
full_df = full_df.drop(['unique_id'], axis=1)

model_names = ['iTransformer', 'TSMixer', 'NHITS', 'PatchTST']
fig, axs = plt.subplots(2, 2, figsize=(15, 15))
bar_width = 0.35
axs = axs.flatten()

for i, ticker in enumerate(tickers):
    df_subset = full_df[(full_df['ticker'] == ticker) & (full_df['metric'] == 'mae')]
    mae_vals = df_subset[model_names].values.flatten()
    df_subset = full_df[(full_df['ticker'] == ticker) & (full_df['metric'] == 'mse')]
    mse_vals = df_subset[model_names].values.flatten()
    indices = np.arange(len(model_names))

    bars_mae = axs[i].bar(indices - bar_width / 2, mae_vals, bar_width, color='skyblue', label='MAE')
    bars_mse = axs[i].bar(indices + bar_width / 2, mse_vals, bar_width, color='orange', label='MSE')

    for bars in [bars_mae, bars_mse]:
        for bar in bars:
            height = bar.get_height()
            axs[i].annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                            xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

    axs[i].set_xticks(indices)
    axs[i].set_xticklabels(model_names, rotation=45)
    axs[i].set_title(ticker)
    axs[i].legend(loc='best')

plt.tight_layout()
plt.show()
for i, ticker in enumerate(tickers):
    nf_preds = predictions_dfs[i]
    stock_data = load_stock_data(ticker, start_date, end_date)

    # Get the mean and standard deviation of the target variable (y)
    y_mean = stock_data['y'].mean()
    y_std = stock_data['y'].std()

    # Rescale the predicted prices
    rescaled_predictions_df = nf_preds.copy()
    rescaled_predictions_df[['iTransformer', 'TSMixer', 'NHITS', 'PatchTST', 'y']] = \
        (nf_preds[['iTransformer', 'TSMixer', 'NHITS', 'PatchTST', 'y']] * y_std) + y_mean
    # Save the rescaled predictions to a CSV file
    now = datetime.now()
    dt_string = now.strftime("%d_%m_%Y %H:%M")
    csv_file = f'predictions/{ticker}_rescaled_predictions_horizon_{horizon}_epochs_{epochs}_{dt_string}.csv'
    rescaled_predictions_df.to_csv(csv_file, index=False, header=True)

    for model_name in model_names:
        plt.figure(figsize=(16, 8))
        plt.plot(stock_data['ds'], stock_data['y'], label='Actual Price', linewidth=3, color='blue')
        plt.plot(rescaled_predictions_df['ds'], rescaled_predictions_df[model_name],
                 label=model_name, linewidth=1, color='red')

        plt.xlabel('Date', fontsize=14)
        plt.ylabel('Price', fontsize=14)
        plt.title(f'{ticker} Stock Price Prediction - {model_name}', fontsize=16)
        plt.grid(axis='y')
        plt.legend(loc='upper left', fontsize=12)
        plt.show()